In [1]:
!pip install tensorflow-tpu -f https://storage.googleapis.com/libtpu-tf-releases/index.html --force

Looking in links: https://storage.googleapis.com/libtpu-tf-releases/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.3/234.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB

In [1]:
!pip install pretty_midi

In [2]:
# imports
import os
import pretty_midi
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, make_scorer

from random import sample
from warnings import catch_warnings
from concurrent.futures import ThreadPoolExecutor
from random import sample

In [3]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.11/dist-packages/jax/__init__.py:31: UserWarning: cloud_tpu_init failed: AttributeError("module 'libtpu' has no attribute 'get_library_path'")
 This a JAX bug; please report an issue at https://github.com/jax-ml/jax/issues
  _warn(f"cloud_tpu_init failed: {exc!r}\n This a JAX bug; please report "


2.18.0


In [4]:
# tensorflow imports
from tensorflow import keras
from keras import Sequential, layers, utils, callbacks
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Input, Normalization, Flatten, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

Device: 
Number of replicas: 8


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# DATA_DIR = "../selectedcomposers"

# google drive folder
DATA_DIR = "/content/drive/MyDrive/selectedcomposers"

composer_list = ["Bach", "Beethoven", "Chopin", "Mozart"]

# get all of the files so we can run a concurrent batch to speed up processing
file_label_pairs = []
for composer in os.listdir(DATA_DIR):
    composer_dir = os.path.join(DATA_DIR, composer)
    if not os.path.isdir(composer_dir):
        continue
    for root, _, files in os.walk(composer_dir):
        for fn in files:
            if fn.lower().endswith((".mid", ".midi")):
                file_label_pairs.append((os.path.join(root, fn), composer))

In [9]:
frequency = 20
seq_size = frequency * 10

# pad the given sequence with zeros if it is too short
def add_padding(seq, x, missing):
  right = np.zeros((x, missing))

  return np.hstack((seq,right))

# get all of the valid sequences from the provided piano roll data.
def get_sequences(piano_data):
  x, y = piano_data.shape
  sequences = []
  if(y < seq_size):
    sequence = add_padding(piano_data, x, seq_size-y)
    sequences.append(sequence)
  else:
    start = 0
    end = 0
    while(end < y):
      end = start + seq_size
      if end > y:
        sequence = piano_data[:,start:y]
        padded = add_padding(sequence, x, (start + seq_size - y))
        sequences.append(padded)
      else:
        sequence = piano_data[:,start:end]
        sequences.append(sequence)
      start += 10

  return sequences

# generate the midi object, remove any invalid notes,
# then get the sequences from the piano roll data
def get_seq_data(file_path):
  midi = pretty_midi.PrettyMIDI(file_path)
  midi.remove_invalid_notes()
  return get_sequences(midi.get_piano_roll(fs=frequency))

In [10]:
midi_data = []

for (path, composer) in file_label_pairs:
  midi_data.append({
    'composer': composer,
    'sequences': get_seq_data(path)
  })

/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [11]:
midi_df = pd.DataFrame(midi_data)

print(f"\nTotal shape: {midi_df.shape}")
print("\nCount by composer:")
print(midi_df['composer'].value_counts())


Total shape: (1627, 2)

Count by composer:
composer
Bach         1024
Mozart        255
Beethoven     212
Chopin        136
Name: count, dtype: int64


In [20]:
# get just the sequence data for each composer and the target row
seq_data = []
for midi in midi_data:
    for seq in midi['sequences']:
        seq_data.append({
            'sequence': np.transpose(seq),
            'composer': midi['composer']
        })

seq_df = pd.DataFrame(seq_data)
seq_df['composer'].value_counts()

,count
composer,
Bach,298248
Beethoven,201956
Mozart,199234
Chopin,57416


In [21]:
rand_sample = 15000
sampled_df = seq_df.groupby("composer", group_keys=False).sample(n=rand_sample, random_state=511)
sampled_df['composer'].value_counts()

,count
composer,
Bach,15000
Beethoven,15000
Chopin,15000
Mozart,15000


In [22]:
X = np.stack(sampled_df['sequence'].values)
y = sampled_df['composer'].values

# shuffle the dataset with replicable seed
indices = np.arange(X.shape[0])
np.random.seed(23)
np.random.shuffle(indices, )

X_shuffled = X[indices]
y_shuffled = y[indices]

In [23]:
# label encode the labels
ohe = OneHotEncoder()
y = ohe.fit_transform(y_shuffled.reshape(-1, 1))
y = y.toarray()
y

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

In [24]:
# split into train/test/val
TEST_TRAIN_SPLIT = 0.8
TEST_VAL_SPLIT = 0.1
total = len(X_shuffled)

# train, test, and val data
X_train = X_shuffled[:int(total * TEST_TRAIN_SPLIT)].copy()
y_train = y[:int(total * TEST_TRAIN_SPLIT)].copy()
X_test = X_shuffled[int(total * (TEST_TRAIN_SPLIT + TEST_VAL_SPLIT)):].copy()
y_test = y[int(total * (TEST_TRAIN_SPLIT + TEST_VAL_SPLIT)):].copy()
X_val = X_shuffled[int(total * TEST_TRAIN_SPLIT):int(total * (TEST_TRAIN_SPLIT + TEST_VAL_SPLIT))].copy()
y_val = y[int(total * TEST_TRAIN_SPLIT):int(total * (TEST_TRAIN_SPLIT + TEST_VAL_SPLIT))].copy()

print("Train / Val / Test shapes:",
      X_train.shape, X_val.shape, X_test.shape)

Train / Val / Test shapes: (48000, 200, 128) (6000, 200, 128) (6000, 200, 128)


In [ ]:
# Save the train, test, and val data to an external file.
np.savez_compressed(
    '../featured datasets/data_splits.npz',
    X_train=X_train, y_train=y_train,
    X_val=X_val,     y_val=y_val,
    X_test=X_test,   y_test=y_test
)

In [ ]:
# Fetch the train, test, and val data and load it in.
data = np.load('data_splits.npz')
X_train = data['X_train']
y_train = data['y_train']
X_val   = data['X_val']
y_val   = data['y_val']
X_test  = data['X_test']
y_test  = data['y_test']

In [26]:
# global training parameters
NUM_EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001

with strategy.scope():
  cnn_model = tf.keras.Sequential([

    # tf.keras.layers.Normalization(axis=None),

    tf.keras.layers.Conv1D(256, kernel_size=3, activation='relu', padding='causal'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(256, kernel_size=3, activation='relu', padding='causal'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu', padding='causal'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu', padding='causal'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', padding='causal'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', padding='causal'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(2),


    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(composer_list), activation='softmax')
  ])

  # Compile the model
  cnn_model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy', 'precision', 'recall', 'f1_score']
  )

In [27]:
# Create the callback
early_stop = EarlyStopping(
    monitor='val_categorical_accuracy',
    patience=5,
    mode='max',
    min_delta=0.0001,
    restore_best_weights=True
)

# setup checkpoint
checkpoint = keras.callbacks.ModelCheckpoint(
  filepath='/content/drive/MyDrive/AAI-511/best_cnn.keras',
  monitor='val_categorical_accuracy',
  mode='max',
  save_best_only=True
)

history = cnn_model.fit(
  X_train, y_train,
  validation_data=(X_val,y_val),
  epochs=NUM_EPOCHS,
  batch_size=BATCH_SIZE,
  callbacks=[early_stop, checkpoint]
)

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - categorical_accuracy: 0.3634 - f1_score: 0.3500 - loss: 1.3821 - precision: 0.5597 - recall: 0.0934 - val_categorical_accuracy: 0.3662 - val_f1_score: 0.3014 - val_loss: 1.2433 - val_precision: 0.7704 - val_recall: 0.0850
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - categorical_accuracy: 0.5941 - f1_score: 0.5800 - loss: 0.9572 - precision: 0.7148 - recall: 0.3969 - val_categorical_accuracy: 0.5052 - val_f1_score: 0.4644 - val_loss: 1.2066 - val_precision: 0.7183 - val_recall: 0.2668
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - categorical_accuracy: 0.6670 - f1_score: 0.6578 - loss: 0.8215 - precision: 0.7449 - recall: 0.5319 - val_categorical_accuracy: 0.5630 - val_f1_score: 0.5439 - val_loss: 1.0924 - val_precision: 0.6881 - val_recall: 0.3772
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - categorical_accuracy: 0.7054 - f1_score: 0.6992 - loss: 0.7378 - precision: 0.7635 - recall: 0.6131 - 

In [28]:
loss, cat_acc, precision, recall, f1 = cnn_model.evaluate(X_test, y_test)

188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - categorical_accuracy: 0.7862 - f1_score: 0.7877 - loss: 0.6195 - precision: 0.8376 - recall: 0.6780


In [29]:
print(f'Loss: {loss}\nAccuracy: {cat_acc}\nPrecision: {precision}\nRecall: {recall},\nF1: {f1}')

Loss: 0.6194760203361511
Accuracy: 0.7861666679382324
Precision: 0.8375540375709534
Recall: 0.6779999732971191,
F1: [0.85863084 0.6921562  0.934495   0.6656626 ]
